# introduction to the vectorizing principle

## part 5b: generating SQL and dataframe transformation code with one syntax - an example pipeline

While pandas is the most popular library for describing data transformations in python, there are more which you will also see in following parts. Relational databases are also a means of highly efficient processing of tabular data. They are typically instructed with a declarative language called SQL which allows for highly sophisticated query engines to first plan how to execute the query. From a large search space of possible execution plans they chose the supposedly best one by using computation cost estimators.

For having a quick look at the data or for searching bugs in previously executed transformation code, it is highly efficient to write ad-hoc SQL scripts by hand. However, for better software engineering practices such as testing, programmatically creating SQL is the preferred option. In fact, it would be ideal to have one syntax from which both dataframe transformation code and SQL can be generated. This is what we show in this part.

The library [pydiverse.transform](https://github.com/pydiverse/pydiverse.transform/) is used here even though it is early stage and still lacks essential features. It is intended as a proof of concept for this idea.

Pandas uses the fluent interface technique chaining functional methods with multiple function calls: `df.assign(x=lambda df: df.int_col*2).loc[lambda df: df.x == 4]`

pydiverse.transform uses the pipe syntax instead, similar to its precursors [dplyr from R](https://dplyr.tidyverse.org/) and [siuba](https://siuba.org/): `df >> mutate(x=λ.int_col*2) >> filter(λ.x == 4)`

pydiverse.transform also supports creating custom verbs with the `@verb` decorator which then can be used within a pipe just like builtin verbs such as `mutate` or `filter`.

Detail: In SQL, joining a table does not add any new columns to the select statement. It rather brings a new table within the namespace that can be used for creating select columns. pydiverse.transform supports this feature by dropping columns with an empty `select()` statement which does not remove them from the namespace for further use in `mutate()` statements.

#### define tasks and helper functions

In [1]:
import os

import pandas as pd
import pydiverse.transform as pdt
import xgboost
import xgboost as xgb
from pydiverse.pipedag import Blob, Flow, Stage, Table, materialize
from pydiverse.transform import aligned, λ
from pydiverse.transform.core.dtypes import String
from pydiverse.transform.core.verbs import (
    alias,
    collect,
    filter,
    left_join,
    mutate,
    select,
)
from pydiverse.transform.eager import PandasTableImpl
from pydiverse.transform.lazy import SQLTableImpl


@pdt.verb
def transmute(tbl, **kwargs):
    return tbl >> select() >> mutate(**kwargs)


@pdt.verb
def trim_all_str(tbl):
    for col in tbl:
        if isinstance(col._.dtype, String):
            tbl[col] = col.strip()
    return tbl


def pk(x: pdt.Table):
    # This is just a placeholder.
    # Ideally there would be a global function in pydiverse transform to
    # get the primary key (and another one to get the table / col name)
    return x.pk


def pk_match(x: pdt.Table, y: pdt.Table):
    return pk(x) == pk(y)


def get_named_tables(tables: list[pdt.Table]) -> dict[str, pdt.Table]:
    return {tbl._impl.name: tbl for tbl in tables}


@materialize(version="1.0.0")
def read_input_data(src_dir="data/pipedag_example_data"):
    return [
        Table(pd.read_csv(os.path.join(src_dir, file)), name=file.removesuffix(".csv.gz"))
        for file in os.listdir(src_dir)
        if file.endswith(".csv.gz")
    ]


@materialize(input_type=SQLTableImpl, lazy=True)
def clean(src_tbls: list[pdt.Table]):
    return [tbl >> trim_all_str() for tbl in src_tbls]


@materialize(input_type=SQLTableImpl, lazy=True, nout=3)
def transform(src_tbls: list[pdt.Table]):
    named_tbls = get_named_tables(src_tbls)
    a = named_tbls["a"]
    b = named_tbls["b"]
    c = named_tbls["c"]

    def join_b(a):
        return a >> left_join(b >> select(), pk_match(a, b))

    new_a = join_b(a) >> mutate(x=b.x)
    new_b = b
    new_c = c

    return new_a, new_b, new_c


@materialize(input_type=SQLTableImpl, lazy=True)
def lazy_features(a: pdt.Table, src_tbls: list[pdt.Table]):
    named_tbls = get_named_tables(src_tbls)
    b = named_tbls["b"]
    return (
        a
        >> left_join(b, pk_match(a, b))
        >> transmute(pk=pk(a), aiige=a.age, y=b.y, z=b.z * 2)
        >> alias("lazy_features")
    )


@materialize(input_type=PandasTableImpl, version="2.3.5")
def eager_features(a: pdt.Table, src_tbls: list[pdt.Table]):
    named_tbls = get_named_tables(src_tbls)
    c = named_tbls["c"]
    return (
        a
        >> left_join(c, pk_match(a, c))
        >> transmute(pk=pk(a), xx=c.x, yy=c.y * 2, zz=c.z + 3)
        >> alias("eager_features")
    )


@materialize(input_type=SQLTableImpl, lazy=True)
def combine_features(features1: pdt.Table, features2: pdt.Table):
    return (
        features1
        >> left_join(
            features2 >> select(-pk(features2)), pk_match(features1, features2)
        )
        >> alias("features")
    )


@materialize(input_type=SQLTableImpl, lazy=True, nout=2)
def train_and_test_set(flat_table: pdt.Table, features: pdt.Table):
    tbl = (
        flat_table
        >> left_join(features, pk_match(flat_table, features))
        >> mutate(row_num=pdt.functions.row_number(arrange=[pk(flat_table)]))
        >> select(-pk(flat_table), -pk(features))
    )

    training_set = (
        tbl
        >> filter(λ.row_num % 10 != 0)
        >> select(-λ.row_num)
        >> alias("training_set")
    )
    test_set = (
        tbl >> filter(λ.row_num % 10 == 0) >> select(-λ.row_num) >> alias("test_set")
    )

    return (training_set, test_set)


@materialize(input_type=pd.DataFrame, version="4.5.8")
def model_training(train_set: pd.DataFrame):
    x = train_set.drop("target", axis=1)
    y = train_set["target"]
    dtrain = xgb.DMatrix(x, label=y)

    params = {"max_depth": 2, "eta": 1, "objective": "binary:logistic"}
    model = xgb.train(params, dtrain)

    return Blob(model, "model")


@aligned(with_="test_set")
def predict(model: xgboost.Booster, test_set: pdt.Table):
    x = test_set >> select(-λ.target) >> collect()

    # Ugly hack to convert new pandas dtypes to numpy dtypes, because xgboost
    # requires numpy dtypes.
    x = x.astype(
        x.dtypes.map(lambda d: d.numpy_dtype if hasattr(d, "numpy_dtype") else d)
    )

    dx = xgb.DMatrix(x)
    predict_col = model.predict(dx)

    return pdt.Table(
        PandasTableImpl("prediction", pd.DataFrame({"prediction": predict_col}))
    ).prediction


@materialize(input_type=PandasTableImpl, version="3.4.5")
def model_evaluation(model: xgboost.Booster, test_set: pdt.Table):
    prediction = predict(model, test_set)  # produces an aligned vector with input
    return (
        test_set
        >> select(λ.target)
        >> mutate(prediction=prediction)
        >> mutate(abs_error=abs(λ.target - λ.prediction))
        >> alias("evaluation")
    )

#### define pipeline:

In [2]:
def get_pipeline():
    with Flow("flow") as flow:
        with Stage("1_raw_input"):
            raw_tbls = read_input_data()

        with Stage("2_clean_input"):
            clean_tbls = clean(raw_tbls)

        with Stage("3_transformed_data"):
            a, b, c = transform(clean_tbls)

        with Stage("4_features"):
            features1 = lazy_features(a, [a, b, c])  # s3.tbls
            features2 = eager_features(a, [a, b, c])  # s3.tbls
            features = combine_features(features1, features2)

        with Stage("5_model"):
            train_set, test_set = train_and_test_set(a, features)
            model = model_training(train_set)

        with Stage("6_evaluation"):
            _ = model_evaluation(model, test_set)

    return flow

#### setup logging:

In [3]:
import logging
from pydiverse.pipedag.util.structlog import setup_logging
setup_logging(log_level=logging.INFO)

#### run pipeline:

In [4]:
flow = get_pipeline()
result = flow.run()
result.visualize()
assert result.successful

2024-08-06 09:20:26.290910 [info     ] Initialized SQL Table Store    [DuckDBTableStore] engine_url=duckdb:////tmp/pipedag/vectorization/db.duckdb schema_prefix= schema_suffix=
2024-08-06 09:20:26.296050 [info     ] Initialized SQL Table Store    [DuckDBTableStore] engine_url=duckdb:////tmp/pipedag/vectorization/db.duckdb schema_prefix= schema_suffix=
2024-08-06 09:20:26.296491 [info     ] Starting IPCServer             [RunContextServer] address=tcp://127.0.0.1:51744
2024-08-06 09:20:26.303510 [info     ] Executing sql                  [DuckDBTableStore]
    [query]
    CREATE SCHEMA IF NOT EXISTS pipedag_metadata
2024-08-06 09:20:26.308873 [info     ] Executing sql                  [DuckDBTableStore]
    [query]
    CREATE SCHEMA IF NOT EXISTS "1_raw_input"
2024-08-06 09:20:26.309606 [info     ] Executing sql                  [DuckDBTableStore]
    [query]
    DROP SCHEMA IF EXISTS "1_raw_input__even" CASCADE
2024-08-06 09:20:26.310429 [info     ] Executing sql                  [Duck

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/martin/micromamba/envs/vectorization/lib/python3.11/site-packages/pydiverse/pipedag/core/ │
│ task.py:168 in run                                                                               │
│                                                                                                  │
│   165 │   │                                                                                      │
│   166 │   │   with run_context, config_context:                                                  │
│   167 │   │   │   try:                                                                           │
│ ❱ 168 │   │   │   │   result, task_context = self._run(inputs)                                   │
│   169 │   │   │   except Exception as e:                                                         │
│   170 │   │   │   │   if config_context._swallow_exceptions:                                     │
│   171 │   │   │   │   │   # PIPEDAG INTERNAL                                                     │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │ config_context = ConfigContext(                                                              │ │
│ │                  │   _config_dict={                                                          │ │
│ │                  │   │   'fail_fast': True,                                                  │ │
│ │                  │   │   'network_interface': '127.0.0.1',                                   │ │
│ │                  │   │   'disable_kroki': False,                                             │ │
│ │                  │   │   'kroki_url': 'https://kroki.io',                                    │ │
│ │                  │   │   'per_user_template': '{id}_{username}',                             │ │
│ │                  │   │   'strict_result_get_locking': True,                                  │ │
│ │                  │   │   'ignore_task_version': False,                                       │ │
│ │                  │   │   'stage_commit_technique': 'READ_VIEWS',                             │ │
│ │                  │   │   'auto_table': [                                                     │ │
│ │                  │   │   │   'pandas.DataFrame',                                             │ │
│ │                  │   │   │   'polars.DataFrame',                                             │ │
│ │                  │   │   │   'pydiverse.transform.Table',                                    │ │
│ │                  │   │   │   'ibis.api.Table',                                               │ │
│ │                  │   │   │   'sqlalchemy.sql.expression.TextClause',                         │ │
│ │                  │   │   │   'sqlalchemy.sql.expression.Selectable'                          │ │
│ │                  │   │   ],                                                                  │ │
│ │                  │   │   'auto_blob': [],                                                    │ │
│ │                  │   │   ... +6                                                              │ │
│ │                  │   },                                                                      │ │
│ │                  │   pipedag_name=None,                                                      │ │
│ │                  │   flow_name='flow',                                                       │ │
│ │                  │   instance_name=None,                                                     │ │
│ │                  │   fail_fast=True,                                                         │ │
│ │                  │   strict_result_get_locking=True,                                         │ │
│ │                  │   ignore_task_version=False,          

2024-08-06 09:20:29.982268 [error    ] Task failed (raised an exception) [Task 'model_evaluation'] task=<Task 'model_evaluation' 0x10806f350 (id: 13)>

2024-08-06 09:20:30.465795 [info     ] Flow visualization             [Flow] url=https://kroki.io/graphviz/svg/eNqtk91ugzAMhe95CsRuN4kApZ1Qd7uHmKbIEIdFS5OKhE3d1Hdf-C_dJrUq3Ngcw-GzcZgoK9i_-c_-t2fqvLspZG0sVpTQCj6pUPvaurKEHOU2OBGDzDP2IHFrtBQs8_Ky0FJX2-AubK8odE90Ui6heM88rpWdCaH_0vtWCKyzpQwsBPc-F1IOhrjhvEid2H0waGrIgtfMO3q_wSNaSAR1jj6TF4CPBvjWdg4MDPljfilwTG0FynBd7ZC1_U_Uv2sLoCcD-uh9C35COYKtKzQT9qQtgJuOayLh6zA5Xwq9Ht9HKB3w1Qab0aDQu1wovN7ir8Gt6E4zlNPUemGBkZHpaLl_LBQFxahFY6lBezE0IaNLC0ZbL6HKm9pOKX6ArMEKrabeT9UlBhCfoZ-4z-E5T1Yx_AMf-g9PftQcdheT5uS4mPZx3UcSNjvqkk2za13cDAXSWhAyZnE71z47ej8HMM8G
2024-08-06 09:20:30.670805 [info     ] Stopped IPCServer              [RunContextServer] address=tcp://127.0.0.1:51744


ValueError: cannot convert NA to integer: Error while type casting for column 'x'

Next: [vectorization06.ipynb](vectorization06.ipynb): many ways to describe data transformations in python